In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Model Development for Molecular Solubility Prediction\n",
    "\n",
    "This notebook demonstrates the model development process for predicting molecular solubility.\n",
    "\n",
    "We'll perform the following tasks:\n",
    "1. Load the processed data with molecular descriptors\n",
    "2. Split the data into training and testing sets\n",
    "3. Train multiple models and optimize hyperparameters\n",
    "4. Evaluate model performance\n",
    "5. Analyze feature importance\n",
    "6. Make predictions on new compounds"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Import libraries\n",
    "import os\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from rdkit import Chem\n",
    "from rdkit.Chem import Draw\n",
    "\n",
    "# Import scikit-learn modules\n",
    "from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error\n",
    "from sklearn.ensemble import RandomForestRegressor\n",
    "from sklearn.linear_model import Ridge, Lasso\n",
    "from sklearn.ensemble import GradientBoostingRegressor\n",
    "\n",
    "# Import from our own modules\n",
    "import sys\n",
    "sys.path.append('..')\n",
    "from src.models.train_model import train_test_split_data, train_random_forest, evaluate_model\n",
    "from src.visualization.visualize import plot_feature_importance, plot_actual_vs_predicted, plot_distributions\n",
    "\n",
    "# Set up plotting\n",
    "%matplotlib inline\n",
    "plt.style.use('seaborn-whitegrid')\n",
    "sns.set_palette('viridis')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Processed Data\n",
    "\n",
    "Let's load the processed data we created in the previous notebook."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Load the processed data\n",
    "features_df = pd.read_csv('../data/processed/molecular_features.csv')\n",
    "\n",
    "# Display the first few rows\n",
    "features_df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Check the shape of the features dataframe\n",
    "print(f\"Feature dataframe shape: {features_df.shape}\")\n",
    "\n",
    "# Check for missing values\n",
    "missing_values = features_df.isnull().sum()\n",
    "print(\"\\nFeatures with missing values:\")\n",
    "print(missing_values[missing_values > 0] if any(missing_values > 0) else \"None\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Prepare Data for Modeling\n",
    "\n",
    "Now we'll split the data into features (X) and target (y), and then into training and testing sets."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Separate features and target\n",
    "X = features_df.drop(['SMILES', 'logS_exp'], axis=1)\n",
    "y = features_df['logS_exp']\n",
    "\n",
    "# Apply feature scaling\n",
    "scaler = StandardScaler()\n",
    "X_scaled = scaler.fit_transform(X)\n",
    "X_scaled = pd.DataFrame(X_scaled, columns=X.columns)\n",
    "\n",
    "# Split data into training and testing sets\n",
    "X_train, X_test, y_train, y_test = train_test_split_data(X_scaled, y, test_size=0.2, random_state=42)\n",
    "\n",
    "print(f\"Training set shape: {X_train.shape}\")\n",
    "print(f\"Testing set shape: {X_test.shape}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Baseline Models\n",
    "\n",
    "Let's train and evaluate several baseline models to establish a performance benchmark."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Define models to evaluate\n",
    "models = {\n",
    "    'Ridge': Ridge(),\n",
    "    'Lasso': Lasso(),\n",
    "    'Random Forest': RandomForestRegressor(random_state=42),\n",
    "    'Gradient Boosting': GradientBoostingRegressor(random_state=42)\n",
    "}\n",
    "\n",
    "# Train and evaluate each model using cross-validation\n",
    "cv_results = {}\n",
    "\n",
    "for name, model in models.items():\n",
    "    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')\n",
    "    rmse_scores = np.sqrt(-cv_scores)\n",
    "    cv_results[name] = {\n",
    "        'mean_rmse': rmse_scores.mean(),\n",
    "        'std_rmse': rmse_scores.std()\n",
    "    }\n",
    "    print(f\"{name}: Mean RMSE = {rmse_scores.mean():.4f} (±{rmse_scores.std():.4f})\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Visualize cross-validation results\n",
    "cv_df = pd.DataFrame({\n",
    "    'Model': list(cv_results.keys()),\n",
    "    'Mean RMSE': [result['mean_rmse'] for result in cv_results.values()],\n",
    "    'Std RMSE': [result['std_rmse'] for result in cv_results.values()]\n",
    "})\n",
    "\n",
    "plt.figure(figsize=(10, 6))\n",
    "sns.barplot(x='Model', y='Mean RMSE', data=cv_df)\n",
    "plt.errorbar(\n",
    "    x=np.arange(len(cv_df)),\n",
    "    y=cv_df['Mean RMSE'],\n",
    "    yerr=cv_df['Std RMSE'],\n",
    "    fmt='none',\n",
    "    color='black',\n",
    "    capsize=5\n",
    ")\n",
    "plt.title('Cross-Validation RMSE for Different Models')\n",
    "plt.ylabel('RMSE')\n",
    "plt.grid(axis='y', alpha=0.3)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Hyperparameter Tuning\n",
    "\n",
    "Based on the baseline results, let's optimize the hyperparameters of the best-performing model."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Define parameter grid for Random Forest\n",
    "rf_param_grid = {\n",
    "    'n_estimators': [100, 200, 300],\n",
    "    'max_depth': [None, 10, 20, 30],\n",
    "    'min_samples_split': [2, 5, 10],\n",
    "    'min_samples_leaf': [1, 2, 4]\n",
    "}\n",
    "\n",
    "# Train Random Forest with hyperparameter tuning\n",
    "rf_model = train_random_forest(X_train, y_train, param_grid=rf_param_grid)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Model Evaluation\n",
    "\n",
    "Now let's evaluate the optimized model on the test set."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Evaluate the optimized model\n",
    "metrics = evaluate_model(rf_model, X_test, y_test)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Make predictions on the test set\n",
    "y_pred = rf_model.predict(X_test)\n",
    "\n",
    "# Plot actual vs predicted values\n",
    "fig = plot_actual_vs_predicted(y_test, y_pred)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Plot distributions of actual and predicted values\n",
    "fig = plot_distributions(y_test, y_pred)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Feature Importance Analysis\n",
    "\n",
    "Let's analyze which molecular descriptors are most important for solubility prediction."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Plot feature importance\n",
    "fig = plot_feature_importance(rf_model, X.columns, top_n=15)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Prediction on New Compounds\n",
    "\n",
    "Finally, let's demonstrate how to use the trained model to predict solubility for new compounds."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Import prediction module\n",
    "from src.models.predict_model import predict_solubility\n",
    "\n",
    "# Example SMILES for new compounds\n",
    "new_smiles = [\n",
    "    'CC(=O)OC1=CC=CC=C1C(=O)O',  # Aspirin\n",
    "    'CN1C=NC2=C1C(=O)N(C(=O)N2C)C',  # Caffeine\n",
    "    'CCC1(CC)C(=O)NC(=O)N(C)C1=O',  # Phenobarbital\n",
    "    'CC(C)CC1=CC=C(C=C1)C(C)C(=O)O',  # Ibuprofen\n",
    "    'CN1C2CCC1CC(C2)OC(=O)C(CO)C3=CC=CC=C3'  # Cocaine\n",
    "]\n",
    "\n",
    "# Make predictions\n",
    "predictions = predict_solubility(rf_model, new_smiles, scaler=scaler)\n",
    "predictions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Visualize the predicted compounds\n",
    "mols = [Chem.MolFromSmiles(smiles) for smiles in predictions['SMILES']]\n",
    "legends = [f\"{Chem.MolToSmiles(mol, isomericSmiles=False)}\\nLogS: {pred:.2f}\" \n",
    "           for mol, pred in zip(mols, predictions['Predicted_Solubility'])]\n",
    "\n",
    "img = Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(300, 300), legends=legends)\n",
    "display(img)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Save Model and Scaler\n",
    "\n",
    "Let's save the trained model and scaler for future use."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Import function to save model\n",
    "from src.models.train_model import save_model\n",
    "\n",
    "# Create directories if they don't exist\n",
    "os.makedirs('../models', exist_ok=True)\n",
    "\n",
    "# Save model\n",
    "save_model(rf_model, '../models/random_forest_model.pkl')\n",
    "\n",
    "# Save scaler\n",
    "save_model(scaler, '../models/scaler.pkl')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "In this notebook, we have:\n",
    "\n",
    "1. Loaded the processed molecular descriptor data\n",
    "2. Prepared the data for modeling by scaling features and splitting into training/testing sets\n",
    "3. Trained and evaluated several baseline models\n",
    "4. Optimized the hyperparameters of the Random Forest model\n",
    "5. Evaluated the optimized model on the test set\n",
    "6. Analyzed feature importance to understand which descriptors are most predictive\n",
    "7. Demonstrated how to make predictions on new compounds\n",
    "8. Saved the trained model and scaler for future use\n",
    "\n",
    "The final model achieved good performance in predicting molecular solubility based on chemical descriptors calculated from molecular structures. This demonstrates a complete machine learning workflow for structure-property relationship modeling."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}